In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
#import pycaret.classification as pc
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
df = dd.read_parquet('..\\full_data\\figma_plan\\full_gender_data.parquet').compute()

# добавляем target, удаляем nan
df = df.merge(pd.read_parquet('..\\full_data\\public_train.pqt',columns=['user_id','is_male'])[['is_male','user_id']], on = 'user_id', how = 'inner')
df = df.loc[~(df['is_male'].isna()) & (df['is_male'] != 'NA')]
df['is_male'] = df['is_male'].astype('int8')

display(df.head())
display(df.info())

,user_id,region_name_0,region_name_1,region_name_2,region_name_3,region_name_4,region_name_5,region_name_6,region_count,city_name_0,...,belnovosti-by.turbopages.org,ngs24-ru.turbopages.org,forum.mfd.ru,yomed.ru,7info.ru,other_url,other_female_urls_frac,other_male_urls_frac,male_probability_by_urls,is_male
0,0,0,0,0,0,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.031088,0.005181,0.000000,0.500491,0
1,1,0,0,0,0,0,0,1,3,0,...,0.0,0.0,0.0,0.0,0.0,0.013372,0.001910,0.000955,0.523982,0
2,2,0,0,0,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.002433,0.000000,0.000000,0.525968,0
3,3,0,0,0,0,0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.010909,0.003636,0.003636,0.519823,1
4,4,0,0,0,0,1,0,0,5,0,...,0.0,0.0,0.0,0.0,0.0,0.011583,0.003861,0.000000,0.510070,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 264326 entries, 0 to 269999
Columns: 3039 entries, user_id to is_male
dtypes: category(26), float32(3009), int32(1), int8(3)
memory usage: 3.0 GB


None

In [3]:
df = df.drop('price',axis=1)
'''nan_urls_frac = df.isna().sum()[df.isna().sum() > 0]
display(nan_urls_frac.head())

display(df.loc[df.price.isna(),['cpe_manufacturer_name_0',
                                'cpe_manufacturer_name_1',
                                'cpe_manufacturer_name_2',
                                'cpe_manufacturer_name_3',
                                'cpe_manufacturer_name_4',
                                'cpe_manufacturer_name_5']])

df.loc[(df['cpe_manufacturer_name_0'] == 0) &
       (df['cpe_manufacturer_name_1'] == 0) &
       (df['cpe_manufacturer_name_2'] == 0) &
       (df['cpe_manufacturer_name_3'] == 0) &
       (df['cpe_manufacturer_name_4'] == 1) &
       (df['cpe_manufacturer_name_5'] == 0), 'price'].dropna().median()'''

"nan_urls_frac = df.isna().sum()[df.isna().sum() > 0]\ndisplay(nan_urls_frac.head())\n\ndisplay(df.loc[df.price.isna(),['cpe_manufacturer_name_0',\n                                'cpe_manufacturer_name_1',\n                                'cpe_manufacturer_name_2',\n                                'cpe_manufacturer_name_3',\n                                'cpe_manufacturer_name_4',\n                                'cpe_manufacturer_name_5']])\n\ndf.loc[(df['cpe_manufacturer_name_0'] == 0) &\n       (df['cpe_manufacturer_name_1'] == 0) &\n       (df['cpe_manufacturer_name_2'] == 0) &\n       (df['cpe_manufacturer_name_3'] == 0) &\n       (df['cpe_manufacturer_name_4'] == 1) &\n       (df['cpe_manufacturer_name_5'] == 0), 'price'].dropna().median()"

In [4]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['is_male','user_id'], axis = 1),df['is_male'], test_size = 0.33, random_state = 42)

Pycaret

In [ ]:
#df = df.drop('user_id',axis=1)
s = pc.setup(df, target = 'is_male', session_id = 123,fold=2)

In [ ]:
best = pc.compare_models(include = ['lightgbm', 'catboost'],fold=2)

Дефолт градиент

In [ ]:
model = GradientBoostingClassifier(random_state=42).fit(x_train,y_train)

In [ ]:
print(f'GINI по полу {2 * roc_auc_score(y_test, model.predict_proba(x_test)[:,1]) - 1:2.3f}')
print(classification_report(y_test, model.predict(x_test)))

Дефолт кетбуст

In [9]:
df.select_dtypes(include=['category']).columns

Index(['region_name_0', 'region_name_1', 'region_name_2', 'region_name_3',
       'region_name_4', 'region_name_5', 'region_name_6', 'city_name_0',
       'city_name_1', 'city_name_2', 'city_name_3', 'city_name_4',
       'city_name_5', 'city_name_6', 'city_name_7', 'city_name_8',
       'city_name_9', 'cpe_manufacturer_name_0', 'cpe_manufacturer_name_1',
       'cpe_manufacturer_name_2', 'cpe_manufacturer_name_3',
       'cpe_manufacturer_name_4', 'cpe_manufacturer_name_5', 'cpe_type_cd_0',
       'cpe_type_cd_1', 'cpe_type_cd_2'],
      dtype='object')

In [12]:
cat_features = df.select_dtypes(include=['category']).columns
model = CatBoostClassifier(random_state=42).fit(x_train,y_train,cat_features.to_list())

Learning rate set to 0.093955
0:	learn: 0.6619685	total: 6.2s	remaining: 1h 43m 13s
1:	learn: 0.6381767	total: 6.92s	remaining: 57m 31s
2:	learn: 0.6173308	total: 7.43s	remaining: 41m 9s
3:	learn: 0.6011103	total: 7.88s	remaining: 32m 42s
4:	learn: 0.5867904	total: 8.34s	remaining: 27m 38s
5:	learn: 0.5757683	total: 8.79s	remaining: 24m 16s
6:	learn: 0.5659184	total: 9.22s	remaining: 21m 47s
7:	learn: 0.5581042	total: 9.61s	remaining: 19m 51s
8:	learn: 0.5507878	total: 10s	remaining: 18m 23s
9:	learn: 0.5445675	total: 10.4s	remaining: 17m 14s
10:	learn: 0.5398094	total: 10.8s	remaining: 16m 14s
11:	learn: 0.5352580	total: 11.3s	remaining: 15m 29s
12:	learn: 0.5314141	total: 11.7s	remaining: 14m 45s
13:	learn: 0.5276451	total: 12.1s	remaining: 14m 12s
14:	learn: 0.5245255	total: 12.5s	remaining: 13m 43s
15:	learn: 0.5217481	total: 12.9s	remaining: 13m 16s
16:	learn: 0.5192871	total: 13.3s	remaining: 12m 51s
17:	learn: 0.5170147	total: 13.7s	remaining: 12m 30s
18:	learn: 0.5148414	total:

In [13]:
print(f'GINI по полу {2 * roc_auc_score(y_test, model.predict_proba(x_test)[:,1]) - 1:2.3f}')
print(classification_report(y_test, model.predict(x_test)))

GINI по полу 0.761
              precision    recall  f1-score   support

           0       0.79      0.79      0.79     42609
           1       0.80      0.80      0.80     44619

    accuracy                           0.80     87228
   macro avg       0.80      0.80      0.80     87228
weighted avg       0.80      0.80      0.80     87228



GridSearchCV

In [ ]:
parameters = {'C': np.linspace(0.5, 1.5, 11)}
clf = GridSearchCV(GradientBoostingClassifier(random_state=42), parameters, cv=5, verbose=5)# n_jobs=-1
clf.fit(df.drop('is_male',axis=1), df['is_male'])